In [1]:
import os
import joblib
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import calendar
from modules.metrics import rmse
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Input, LSTM, ConvLSTM2D, Dense,
    Dropout, SpatialDropout2D, MaxPooling2D, BatchNormalization,
    TimeDistributed, LeakyReLU, Flatten
)
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
print(gpus)

1 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 資料分割

In [3]:
# # split data by month
# def split_data_by_month(data_path):
#     train_files, val_files, test_files = [], [], []
#     file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv

#     for file_name in file_list:
#         datetime = file_name.split('-')[0]
#         year = int(datetime[0:4])
#         month = int(datetime[4:6])
#         day = int(datetime[6:8])
#         # 取得當月的天數
#         days_in_month = calendar.monthrange(year, month)[1]
#         part1_end = int(days_in_month * 0.7)
#         part2_end = int(days_in_month * 0.85)

#         file_path = os.path.join(data_path, file_name)
#         if day <= part1_end:
#             train_files.append(file_path)
#         elif day <= part2_end:
#             val_files.append(file_path)
#         else:
#             test_files.append(file_path)

#     return (train_files, val_files, test_files)


# split data by sequence
def split_data_by_sequence(data_path):
    train_files, val_files, test_files = [], [], []
    file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv

    train_cases_to_exclude = {'20210116', '20210530', '20210825', '20210722', '20220904'}

    for file_name in file_list:
        datetime = file_name.split('-')[0]

        file_path = os.path.join(data_path, file_name)
        # 加入至測試集
        if any(datetime.startswith(case) for case in train_cases_to_exclude):
            test_files.append(file_path)
            continue # 加入測試集後跳過當前迴圈，為了不將同一檔案加入訓練集或驗證集
        if datetime.startswith(('2021', '202201', '202202', '202203', '202204', '202205', '202206')):
            train_files.append(file_path)
        if datetime.startswith(('202207', '202208', '202209')):
            val_files.append(file_path)
        if datetime.startswith(('202210', '202211', '202212')):
            test_files.append(file_path)

    return (train_files, val_files, test_files)

# mont and sequence data intersect
# def intersect_test_files(data_path):
#     test_files = []
#     file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv

#     for file_name in file_list:
#         datetime = file_name.split('-')[0]

#         file_path = os.path.join(data_path, file_name)
        
#         if not datetime.startswith(('202210', '202211', '202212')):
#             continue

#         datetime = file_name.split('-')[0]
#         year = int(datetime[0:4])
#         month = int(datetime[4:6])
#         day = int(datetime[6:8])
#         # 取得當月的天數
#         days_in_month = calendar.monthrange(year, month)[1]
#         part1_end = int(days_in_month * 0.7)
#         part2_end = int(days_in_month * 0.85)
#         file_path = os.path.join(data_path, file_name)

#         if day > part2_end:
#             test_files.append(file_path)

#     return test_files

# 資料縮放器

In [4]:
def fit_scaler_on_training_data(column_name, csv_files):
    """
    預先處理和縮放數據集，將經緯度縮放到同一範圍。
    """
    lat_list = []
    lng_list = []

    for csv_file_path in csv_files:
        data = pd.read_csv(csv_file_path, encoding='utf-8',
                           dtype={'FileName': str, 'Date': str, 'Time': str, 'DateTime': str})

        lat = data[column_name[0]]
        lng = data[column_name[1]]

        lat_list.extend(lat.values)
        lng_list.extend(lng.values)

    # 將經緯值進行縮放
    combined_data = np.vstack([lat_list, lng_list]).T
    scaler = MinMaxScaler()
    scaler.fit(combined_data)

    return scaler

# 資料生成器

In [5]:
def csv_sliding_window_generator(csv_files, column_name, scaler, window_size, step_size):
    """
    預先處理CSV的滑動窗口，並將其生成。
    """

    for csv_file_path in csv_files:
        csv_file = os.path.basename(csv_file_path)
        data = pd.read_csv(csv_file_path, encoding='utf-8',
                           dtype={'FileName': str, 'Date': str, 'Time': str, 'DateTime': str})
        
        # 提取 X（經過縮放）和 y（未縮放）
        lat = data[column_name[0]]
        lng = data[column_name[1]]
        combined_data = np.vstack([lat.values, lng.values]).T  # 經緯度差異合併

        # 使用scaler對X進行縮放
        scaled_data = scaler.transform(combined_data)
        
        x, y_lat, y_lng = [], [], []
        for i in range(0, len(scaled_data) - window_size, step_size):
            x.append(scaled_data[i:i + window_size])

            y_scaled = scaled_data[i + window_size]
            y_lat.append(y_scaled[0])  # 縮放緯度
            y_lng.append(y_scaled[1])  # 縮放經度

        # y_lat_sample, y_lng_sample 都是縮放過的經緯度值
        for x_sample, y_lat_sample, y_lng_sample in zip(x, y_lat, y_lng):
            yield np.array(x_sample, dtype=np.float32), {  
                'lstm_lat_output': np.array([y_lat_sample], dtype=np.float32),  # 緯度差標籤不縮放
                'lstm_lng_output': np.array([y_lng_sample], dtype=np.float32)   # 經度差標籤不縮放
            }

def create_csv_sliding_window_dataset(csv_files, column_name, scaler, window_size, step_size):
    """
    使用滑動窗口生成TensorFlow Dataset，避免在生成過程中處理時間序列。
    """
    dataset = tf.data.Dataset.from_generator(
        lambda: csv_sliding_window_generator(csv_files, column_name, scaler, window_size, step_size),
        output_signature=(
            tf.TensorSpec(shape=(window_size, 2), dtype=tf.float32), # X shape: (window_size, 2)
            {
                'lstm_lat_output': tf.TensorSpec(shape=(1,), dtype=tf.float32),  # 緯度標籤
                'lstm_lng_output': tf.TensorSpec(shape=(1,), dtype=tf.float32)   # 經度標籤
            },
        )
        
    )
    return dataset


In [6]:
split_data_mode = 'sequence'  # 'month' or 'sequence' or 'old_dataset'
cells_csv_path = r'H:\cell_data_processed\cells'

if split_data_mode == 'month':
    train_files, val_files, test_files = split_data_by_month(cells_csv_path)
else:
    train_files, val_files, test_files = split_data_by_sequence(cells_csv_path)

# 使用共同測試資料區間
# test_files = intersect_test_files(cells_csv_path)
print('train_files:', len(train_files))
print('val_files:', len(val_files))
print('test_files:', len(test_files))

train_files: 60330
val_files: 10192
test_files: 10251


In [7]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 3  # 窗口大小
step_size = 1  # 步長

# 創建經緯度滑動窗口數據集
# scaler = fit_scaler_on_training_data(column_name, train_files)  # 擬合scaler
# joblib.dump(scaler, f'config/{split_data_mode}/lstm_multitask_scaler.gz')  # 保存scaler
scaler = joblib.load(f'config/{split_data_mode}/lstm_multitask_scaler.gz')  # 載入scaler

# 創建csv數據集
train_dataset = create_csv_sliding_window_dataset(
    train_files, column_name, scaler, window_size, step_size)

val_dataset = create_csv_sliding_window_dataset(
    val_files, column_name, scaler, window_size, step_size)

In [8]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(3, 2), dtype=tf.float32, name=None), {'lstm_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'lstm_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})
(TensorSpec(shape=(3, 2), dtype=tf.float32, name=None), {'lstm_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'lstm_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})


In [ ]:
# for x, y in train_dataset.take(1):
#     print(f'X shape: {x.shape}')
#     print(f'Lat y shape: {y["lstm_lat_output"].shape}')
#     print(f'Lng y shape: {y["lstm_lng_output"].shape}')

# for x, y in val_dataset.take(1):
#     print(f'X shape: {x.shape}')
#     print(f'Lat y shape: {y["lstm_lat_output"].shape}')
#     print(f'Lng y shape: {y["lstm_lng_output"].shape}')

# 建立模型

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5,
                               verbose=1, mode='auto', restore_best_weights=True)

checkpoint = ModelCheckpoint(os.path.join(
    os.getcwd(), 'weights', split_data_mode, 'lstm_multitask-scaler_y', 'lstm_mt_3-1_e{epoch:02d}v{val_loss:.4f}'),
    monitor='val_loss', save_best_only=True)

# Multi-task LSTM model
n_in = window_size
n_features = 2
inputs = Input(shape=(n_in, n_features), name='lstm_input')

lstm_1 = LSTM(32, activation=LeakyReLU(),
              return_sequences=True, name='lstm_1')(inputs)
lstm_1 = BatchNormalization(name='lstm_1-bn_1')(lstm_1)
dropout_1 = Dropout(0.2, name='lstm_1-dropout_1')(lstm_1)

lstm_2 = LSTM(64, activation=LeakyReLU(),
              return_sequences=True, name='lstm_2')(dropout_1)
lstm_2 = BatchNormalization(name='lstm_2-bn_2')(lstm_2)
dropout_2 = Dropout(0.2, name='lstm_2-dropout_2')(lstm_2)

lstm_3 = LSTM(128, activation=LeakyReLU(),
              return_sequences=False, name='lstm_3')(dropout_2)
lstm_3 = BatchNormalization(name='lstm_3-bn_3')(lstm_3)
dropout_3 = Dropout(0.2, name='lstm_3-dropout_3')(lstm_3)

dense_1 = Dense(64, kernel_regularizer=l2(
    0.01), activation='linear', name='lstm_dense')(dropout_3)
dropout_4 = Dropout(0.2, name='lstm_dense-dropout')(dense_1)

lat_output = Dense(1, activation='linear', name='lstm_lat_output')(dropout_4)
lng_output = Dense(1, activation='linear', name='lstm_lng_output')(dropout_4)

lstm_model = Model(inputs=inputs, outputs=[
                   lat_output, lng_output], name='lstm_multi_task')

lstm_model.compile(optimizer=Adam(learning_rate=0.0001),
                   loss={'lstm_lat_output': 'mse',
                         'lstm_lng_output': 'mse'},
                   loss_weights={'lstm_lat_output': 1.0,
                                 'lstm_lng_output': 1.0},
                   metrics={'lstm_lat_output': ['mse', rmse, 'mae'],
                            'lstm_lng_output': ['mse', rmse, 'mae']})

print(lstm_model.summary())

Model: "lstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 3, 2)]       0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 3, 32)        4480        ['lstm_input[0][0]']             
                                                                                                  
 lstm_1-bn_1 (BatchNormalizatio  (None, 3, 32)       128         ['lstm_1[0][0]']                 
 n)                                                                                               
                                                                                                  
 lstm_1-dropout_1 (Dropout)     (None, 3, 32)        0           ['lstm_1-bn_1[0][0]

In [12]:
# 設定訓練參數
batch_size = 32
epochs = 50

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
train_dataset = train_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
# 開始訓練模型
history = lstm_model.fit(
    train_dataset,
    validation_data=val_dataset,  # 傳入驗證集
    epochs=epochs,
    callbacks=[early_stopping, checkpoint],
    verbose=1  # 訓練過程中打印進度
)

Epoch 1/50
   6541/Unknown - 314s 47ms/step - loss: 0.8969 - lstm_lat_output_loss: 0.1747 - lstm_lng_output_loss: 0.1737 - lstm_lat_output_mse: 0.1747 - lstm_lat_output_rmse: 0.3132 - lstm_lat_output_mae: 0.2469 - lstm_lng_output_mse: 0.1737 - lstm_lng_output_rmse: 0.3152 - lstm_lng_output_mae: 0.2480

INFO:tensorflow:Assets written to: f:\yucheng對流胞\PythonCode\master_thesis\1.ffnn_vs_time_series_model\time_series_model\weights\sequence\new\lstm_multitask-scaler_y\lstm_mt_3-1_e01v0.1991\assets


INFO:tensorflow:Assets written to: f:\yucheng對流胞\PythonCode\master_thesis\1.ffnn_vs_time_series_model\time_series_model\weights\sequence\new\lstm_multitask-scaler_y\lstm_mt_3-1_e01v0.1991\assets


6542/6542 [==============================] - 371s 56ms/step - loss: 0.8969 - lstm_lat_output_loss: 0.1747 - lstm_lng_output_loss: 0.1737 - lstm_lat_output_mse: 0.1747 - lstm_lat_output_rmse: 0.3132 - lstm_lat_output_mae: 0.2468 - lstm_lng_output_mse: 0.1737 - lstm_lng_output_rmse: 0.3151 - lstm_lng_output_mae: 0.2480 - val_loss: 0.1991 - val_lstm_lat_output_loss: 0.0024 - val_lstm_lng_output_loss: 0.0029 - val_lstm_lat_output_mse: 0.0024 - val_lstm_lat_output_rmse: 0.0452 - val_lstm_lat_output_mae: 0.0395 - val_lstm_lng_output_mse: 0.0029 - val_lstm_lng_output_rmse: 0.0493 - val_lstm_lng_output_mae: 0.0437
Epoch 2/50
6542/6542 [==============================] - ETA: 0s - loss: 0.0509 - lstm_lat_output_loss: 0.0078 - lstm_lng_output_loss: 0.0077 - lstm_lat_output_mse: 0.0078 - lstm_lat_output_rmse: 0.0774 - lstm_lat_output_mae: 0.0683 - lstm_lng_output_mse: 0.0077 - lstm_lng_output_rmse: 0.0782 - lstm_lng_output_mae: 0.0682

INFO:tensorflow:Assets written to: f:\yucheng對流胞\PythonCode\master_thesis\1.ffnn_vs_time_series_model\time_series_model\weights\sequence\new\lstm_multitask-scaler_y\lstm_mt_3-1_e02v0.0019\assets


...

INFO:tensorflow:Assets written to: f:\yucheng對流胞\PythonCode\master_thesis\1.ffnn_vs_time_series_model\time_series_model\weights\sequence\new\lstm_multitask-scaler_y\lstm_mt_3-1_e04v0.0015\assets


6542/6542 [==============================] - 364s 56ms/step - loss: 0.0138 - lstm_lat_output_loss: 0.0068 - lstm_lng_output_loss: 0.0067 - lstm_lat_output_mse: 0.0068 - lstm_lat_output_rmse: 0.0697 - lstm_lat_output_mae: 0.0639 - lstm_lng_output_mse: 0.0067 - lstm_lng_output_rmse: 0.0706 - lstm_lng_output_mae: 0.0637 - val_loss: 0.0015 - val_lstm_lat_output_loss: 3.9630e-04 - val_lstm_lng_output_loss: 2.1315e-04 - val_lstm_lat_output_mse: 3.9630e-04 - val_lstm_lat_output_rmse: 0.0180 - val_lstm_lat_output_mae: 0.0152 - val_lstm_lng_output_mse: 2.1315e-04 - val_lstm_lng_output_rmse: 0.0129 - val_lstm_lng_output_mae: 0.0106
Epoch 5/50
6542/6542 [==============================] - 376s 57ms/step - loss: 0.0135 - lstm_lat_output_loss: 0.0067 - lstm_lng_output_loss: 0.0065 - lstm_lat_output_mse: 0.0067 - lstm_lat_output_rmse: 0.0685 - lstm_lat_output_mae: 0.0632 - lstm_lng_output_mse: 0.0065 - lstm_lng_output_rmse: 0.0692 - lstm_lng_output_mae: 0.0628 - val_loss: 0.0019 - val_lstm_lat_output

INFO:tensorflow:Assets written to: f:\yucheng對流胞\PythonCode\master_thesis\1.ffnn_vs_time_series_model\time_series_model\weights\sequence\new\lstm_multitask-scaler_y\lstm_mt_3-1_e06v0.0015\assets


INFO:tensorflow:Assets written to: f:\yucheng對流胞\PythonCode\master_thesis\1.ffnn_vs_time_series_model\time_series_model\weights\sequence\new\lstm_multitask-scaler_y\lstm_mt_3-1_e06v0.0015\assets


6542/6542 [==============================] - 378s 58ms/step - loss: 0.0134 - lstm_lat_output_loss: 0.0067 - lstm_lng_output_loss: 0.0065 - lstm_lat_output_mse: 0.0067 - lstm_lat_output_rmse: 0.0679 - lstm_lat_output_mae: 0.0630 - lstm_lng_output_mse: 0.0065 - lstm_lng_output_rmse: 0.0684 - lstm_lng_output_mae: 0.0624 - val_loss: 0.0015 - val_lstm_lat_output_loss: 2.1936e-04 - val_lstm_lng_output_loss: 2.7070e-04 - val_lstm_lat_output_mse: 2.1936e-04 - val_lstm_lat_output_rmse: 0.0137 - val_lstm_lat_output_mae: 0.0116 - val_lstm_lng_output_mse: 2.7070e-04 - val_lstm_lng_output_rmse: 0.0148 - val_lstm_lng_output_mae: 0.0123
Epoch 7/50
6542/6542 [==============================] - 359s 55ms/step - loss: 0.0133 - lstm_lat_output_loss: 0.0066 - lstm_lng_output_loss: 0.0064 - lstm_lat_output_mse: 0.0066 - lstm_lat_output_rmse: 0.0673 - lstm_lat_output_mae: 0.0626 - lstm_lng_output_mse: 0.0064 - lstm_lng_output_rmse: 0.0676 - lstm_lng_output_mae: 0.0620 - val_loss: 0.0017 - val_lstm_lat_output

# 評估模型

In [8]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 3  # 窗口大小
step_size = 1  # 步長

# 創建csv數據集
test_dataset = create_csv_sliding_window_dataset(
    test_files, column_name, scaler, window_size, step_size)

In [9]:
test_dataset.element_spec

(TensorSpec(shape=(3, 2), dtype=tf.float32, name=None),
 {'lstm_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
  'lstm_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})

In [10]:
# 設定訓練參數
batch_size = 32

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
test_dataset = test_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
# if split_data_mode == 'month':
#     model_path = os.path.join(os.getcwd(), r'weights\month\lstm_multitask\lstm_mt_diff2-1_e18v1.2658')
if split_data_mode == 'sequence':
    model_path = os.path.join(os.getcwd(), r'weights\sequence\lstm_multitask-scaler_y\lstm_mt_3-1_e06v0.0015')
    
if os.path.exists(model_path):
    lstm_model = load_model(model_path, custom_objects={'rmse': rmse})
    print('Load model successfully!')
    print(lstm_model.summary())

Load model successfully!
Model: "lstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 3, 2)]       0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 3, 32)        4480        ['lstm_input[0][0]']             
                                                                                                  
 lstm_1-bn_1 (BatchNormalizatio  (None, 3, 32)       128         ['lstm_1[0][0]']                 
 n)                                                                                               
                                                                                                  
 lstm_1-dropout_1 (Dropout)     (None, 3, 32)        0     

In [ ]:
from haversine import haversine

distances = []
lat_errors = []
lon_errors = []

feature_columns = ['Latitude', 'Longitude']
window_size = 3  # 輸入三步
output_steps = 1  # 預測下一步(t4)
step_size = 1

for csv_file in test_files:
    csv_file_path = os.path.join(cells_csv_path, csv_file)
    df = pd.read_csv(csv_file_path)

    if len(df) < window_size + output_steps:
        continue  # 資料不夠就跳過

    lats = df['Latitude'].values
    lons = df['Longitude'].values
    x = df[feature_columns].values
    scaled_data = scaler.transform(x)

    inputs = []
    for i in range(0, len(scaled_data) - window_size, step_size):
        input_sample = scaled_data[i:i + window_size]
        inputs.append(input_sample)

    inputs = np.array(inputs)  # shape: (samples, 3, 2)

    # 預測
    pred_lats, pred_lons = lstm_model.predict(inputs)  # shape: (samples, 1), (samples, 1)

    # 反縮放
    pred_lats = scaler.inverse_transform(np.hstack((pred_lats, np.zeros_like(pred_lats))))[:, 0]
    pred_lons = scaler.inverse_transform(np.hstack((np.zeros_like(pred_lons), pred_lons)))[:, 1]

    for i in range(len(pred_lats)):
        # 預測的是 i 對應的 t(i+3)
        real_lat = lats[i + window_size]
        real_lon = lons[i + window_size]
        pred_lat = pred_lats[i]
        pred_lon = pred_lons[i]

        error_lat = abs(real_lat - pred_lat)
        error_lon = abs(real_lon - pred_lon)
        lat_errors.append(error_lat)
        lon_errors.append(error_lon)

        dist = haversine((real_lat, real_lon), (pred_lat, pred_lon))
        distances.append(dist)

# 統計結果
avg_distance = np.mean(distances)
avg_lat_error = np.mean(lat_errors)
avg_lon_error = np.mean(lon_errors)

mse_lat = np.mean(np.square(lat_errors))
mse_lon = np.mean(np.square(lon_errors))
rmse_lat = np.sqrt(np.mean(np.square(lat_errors)))
rmse_lon = np.sqrt(np.mean(np.square(lon_errors)))
mae_lat = avg_lat_error  # 就是 mean(abs(...))
mae_lon = avg_lon_error

print(f'=== 第一步預測 (t+1) 經緯度誤差 ===')
print(f"Latitude MSE：{mse_lat:.6f} 度")
print(f"Longitude MSE：{mse_lon:.6f} 度\n")
print(f"Latitude MAE：{mae_lat:.6f} 度")
print(f"Longitude MAE：{mae_lon:.6f} 度\n")
print(f"Latitude RMSE：{rmse_lat:.6f} 度")
print(f"Longitude RMSE：{rmse_lon:.6f} 度\n")
print(f"Average Haversine distance error: {avg_distance:.6f} km\n")


1/1 [==============================] - 0s 32ms/step
[25.5249   25.510256]
[25.5249   25.510256]
1/1 [==============================] - 0s 24ms/step
[25.59066]
1/1 [==============================] - 0s 21ms/step
[25.318819 25.302427 25.278898 25.247866]
[25.318819 25.302427 25.278898 25.247866]
[25.318819 25.302427 25.278898 25.247866]
[25.318819 25.302427 25.278898 25.247866]
1/1 [==============================] - 0s 32ms/step
[25.250164]
1/1 [==============================] - 0s 17ms/step
[25.209452]
1/1 [==============================] - 0s 18ms/step
[25.11835  25.109869 25.102371 25.09511  25.084219 25.07078  25.059355
 25.054113]
[25.11835  25.109869 25.102371 25.09511  25.084219 25.07078  25.059355
 25.054113]
[25.11835  25.109869 25.102371 25.09511  25.084219 25.07078  25.059355
 25.054113]
[25.11835  25.109869 25.102371 25.09511  25.084219 25.07078  25.059355
 25.054113]
[25.11835  25.109869 25.102371 25.09511  25.084219 25.07078  25.059355
 25.054113]
[25.11835  25.109869 25.10

In [ ]:
# evaluation = lstm_model.evaluate(test_dataset)

# print("=== 經緯度差（difference）評估 ===")

# print(f'Total Loss: {evaluation[0]:.5f}')         # 總損失

# print(f'Latitude Loss: {evaluation[1]:.5f}')       # 緯度 MSE
# print(f'Longitude Loss: {evaluation[2]:.5f}')      # 經度 MSE

# print(f'Latitude MSE: {evaluation[3]:.5f}')       # 緯度 MSE
# print(f'Longitude MSE: {evaluation[6]:.5f}')      # 經度 MSE

# print(f'Latitude RMSE: {evaluation[4]:.5f}')       # 緯度 RMSE
# print(f'Longitude RMSE: {evaluation[7]:.5f}')      # 經度 RMSE

# print(f'Latitude MAE: {evaluation[5]:.5f}')       # 緯度 MAE
# print(f'Longitude MAE: {evaluation[8]:.5f}')      # 經度 MAE

# import math

# # ============= 計算誤差(公里) =============
# def calculate_mae_distance(lat_mae: float, lon_mae: float, latitude: float = 25.071182):
#     lat_km = lat_mae * 111
#     lon_km = lon_mae * 111 * math.cos(math.radians(latitude))
#     return math.sqrt(lat_km**2 + lon_km**2)

# # 範例數據：經度差 MAE = 0.02，緯度差 MAE = 0.01，五分山雷達站的緯度 = 23.5
# lat_mae = evaluation[5]
# lon_mae = evaluation[8]
# # 固定緯度
# # 五分山雷達站的經緯度
# center_lat = 25.071182
# center_lon = 121.781205

# mae_distance = calculate_mae_distance(lat_mae, lon_mae, center_lat)
# print(f"Average distance error (via MAE): {mae_distance:.2f} km")

992/992 [==============================] - 63s 62ms/step - loss: 0.9638 - lstm_lat_output_loss: 0.6756 - lstm_lng_output_loss: 0.0691 - lstm_lat_output_mse: 0.6756 - lstm_lat_output_rmse: 0.7446 - lstm_lat_output_mae: 0.6601 - lstm_lng_output_mse: 0.0691 - lstm_lng_output_rmse: 0.2448 - lstm_lng_output_mae: 0.2215
=== 經緯度差（difference）評估 ===
Total Loss: 0.96380
Latitude Loss: 0.67560
Longitude Loss: 0.06905
Latitude MSE: 0.67560
Longitude MSE: 0.06905
Latitude RMSE: 0.74457
Longitude RMSE: 0.24481
Latitude MAE: 0.66010
Longitude MAE: 0.22150
Average distance error (via MAE): 76.58 km


In [14]:
# from haversine import haversine

# distances = []
# lat_errors = []
# lon_errors = []

# for csv_file in test_files:
#     df = pd.read_csv(csv_file)

#     # 真實經緯度序列
#     lats = df['Latitude'].values
#     lons = df['Longitude'].values

#     # 差分計算
#     delta_lat = np.diff(lats)
#     delta_lon = np.diff(lons)
#     combined_data = np.vstack([delta_lat, delta_lon]).T

#     # 特徵縮放
#     scaled_data_diff = scaler.transform(combined_data)

#     # 準備滑動窗口資料
#     inputs = []
#     for i in range(len(scaled_data_diff) - window_size):
#         input_sample = [scaled_data_diff[i], scaled_data_diff[i + 1]]
#         inputs.append(input_sample)

#     inputs = np.array(inputs)

#     # 模型預測差值
#     pred_lats_diff, pred_lons_diff = lstm_model.predict(inputs)

#     # 差值還原回經緯度
#     pred_lats = []
#     pred_lons = []
#     for i, (dlat, dlon) in enumerate(zip(pred_lats_diff, pred_lons_diff)):
#         base_lat = lats[i + 2]
#         base_lon = lons[i + 2]
#         pred_lat = base_lat + dlat.item()
#         pred_lon = base_lon + dlon.item()
#         pred_lats.append(pred_lat)
#         pred_lons.append(pred_lon)

#     # 計算實際誤差
#     for i in range(len(pred_lats)):
#         real_lat = lats[i + 3]
#         real_lon = lons[i + 3]
#         pred_lat = pred_lats[i]
#         pred_lon = pred_lons[i]

#         lat_errors.append(real_lat - pred_lat)
#         lon_errors.append(real_lon - pred_lon)
#         distances.append(haversine((real_lat, real_lon), (pred_lat, pred_lon)))

# # === 評估指標 ===
# lat_errors = np.array(lat_errors)
# lon_errors = np.array(lon_errors)
# distances = np.array(distances)

# # MAE
# lat_mae = np.mean(np.abs(lat_errors))
# lon_mae = np.mean(np.abs(lon_errors))

# # MSE
# lat_mse = np.mean(lat_errors ** 2)
# lon_mse = np.mean(lon_errors ** 2)

# # RMSE
# lat_rmse = np.sqrt(lat_mse)
# lon_rmse = np.sqrt(lon_mse)

# # 結果輸出
# print("=== 經緯度位置評估 ===")
# print(f"Latitude MAE: {lat_mae:.6f} 度")
# print(f"Longitude MAE: {lon_mae:.6f} 度")
# print(f"Latitude MSE: {lat_mse:.6f}")
# print(f"Longitude MSE: {lon_mse:.6f}")
# print(f"Latitude RMSE: {lat_rmse:.6f}")
# print(f"Longitude RMSE: {lon_rmse:.6f}")
# print(f"Average Haversine distance: {np.mean(distances):.2f} km")
